# Scope of work

1) Import all necessary libraries and modules.  
2) First we need to obtain inforamtion about all available cars from the first main pages.  
3) Create a SQL database and export there gathered info about cars from the main pages.  
4) Then we will call module dealers_cars to acquire links to every dealer's list of cars from all main pages.  
5) Now we will repeat the same procedure as in the 1st step but to the every dealer's cars list.  
6) Add extracted data to a created SQL database

## 1. Imort of libraries and modules

In [1]:
import main_pages
import parsing
import marks
import dataframe
import sql_db
import cars_scraper
from datetime import datetime

## 2. Acquring cars' information from the first main pages

Here we call the 1st module 'parsing' to scrap data from the main pages of the website. Then we store all gathered information about cars into corresponding lists.

In [2]:
# Link to the main webpage
url = 'https://www.autoscout24.com/lst?atype=C&desc=0&sort=standard&source=homepage_search-mask&ustate=N%2CU'

The module **main_pages** collects all URLs of the main pages from the website autoscout24.com

In [3]:
all_pages = main_pages.pages_urls(url)

With a help of **parsing** module we scrap info about cars from main pages (20).

In [4]:
start = datetime.now()
cars, characteristics, prices, locations = parsing.cars_info(all_pages)
end = datetime.now()
print('Total time :', end-start)

Total time : 0:00:28.083891


Here we call the module **marks** in order to extract all existing car marks from the website. Afterwards we will replace
spaces in marks' names into dashes '-'

In [5]:
marks_menu = marks.all_marks(url)

Here we call the module **dataframe** in order to gather all info about cars into one dataframe

In [6]:
df = dataframe.df_construct(marks_menu, cars, characteristics, prices, locations)

In [7]:
df

,mark,model,mileage,transmission,registration,fuel,power,location,price
0,BMW,i7,5000,Automatic,11/2023,Electric,658,AT,169900
1,Jeep,Renegade,124904,Automatic,07/2018,Gasoline,140,BE,11400
2,Audi,R8,88150,Automatic,04/2012,Gasoline,430,DE,64900
3,Mercedes-Benz,GLC 220,283000,Automatic,03/2018,Diesel,163,BE,20500
4,Renault,Megane,5834,Automatic,03/2022,Gasoline,300,BE,38889
...,...,...,...,...,...,...,...,...,...
395,Porsche,992,199,Manual,03/2023,Gasoline,551,DE,449900
396,Mazda,3,131450,Manual,04/2015,Diesel,150,DE,10590
397,BMW,318,112359,Manual,07/2018,Diesel,136,BE,14400
398,Audi,S6,37450,Automatic,10/2020,Diesel,349,DE,59940


## 3. Creating a SQL database and exporting parsed data there from the main pages

Here we connect to another module **sql_db**. This module connects to a PostgreSQL database *autoscout*. In this database there is a schema *autoscout* which contains the main table *cars*.

In [8]:
sql_db.connect(df,'replace')

## 4. Acquring cars' information from the all the car dealers and storing into a SQL database

In [9]:
import multiprocessing

In [10]:
start = datetime.now()

for page in all_pages:
    cars_scraper.parser(page, marks_menu)

end = datetime.now()
print('Total time :', end-start)

Total time : 1:23:24.669260


Total time :1:28:41.573803

In [ ]:
start = datetime.now()
def process_page(page, marks_menu):
    cars_scraper.parser(page, marks_menu)
    
num_processes = len(all_pages)  # Number of processes equals the number of pages

with multiprocessing.Pool(num_processes) as pool:
    pool.starmap(process_page, [(page, marks_menu) for page in all_pages])
    
end = datetime.now()
print('Total time :', end-start)